In [11]:
'''
~~~~~~~~~~~ Start of Quantum Machine Learning Script ~~~~~~~~~~~

Ideas/Improvements:
- This model uses direct eigenvectors of the Hamiltonian for training data. Instead, it
is possible to use optimized angles of each nuclear configuration instead, and generate
a state_vector out of this (which is entirely supported by Pennylane) for training data.
However, this requires the Adapt-VQE gate selection protocol to be run first to select
all necessary quantum gates so all training data wavefunctions have the same number of
gate parameters

'''

'\n~~~~~~~~~~~ Start of Quantum Machine Learning Script! ~~~~~~~~~~~\n\nIdeas/Improvements:\n- This model uses direct eigenvectors of the Hamiltonian for training data. Instead, it\nis possible to use optimized angles of each nuclear configuration instead, and generate\na state_vector out of this (which is entirely supported by Pennylane) for training data.\nHowever, this requires the Adapt-VQE gate selection protocol to be run first to select\nall necessary quantum gates so all training data wavefunctions have the same number of\ngate parameters\n\n'

In [1]:
# Imports and Relevant Defines:

# Jax needs more precision to calculate quantum data,
# so ensure the environment variable is set before importing JAX
import os
os.environ['JAX_ENABLE_X64'] = 'True'

#import autohf as hf
import pennylane.numpy as np
import pennylane as qml
import jax
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import jax.numpy as jnp
# import tailgating as tg
import scipy as sc
import optax # is a gradient-processing library for jax
from pennylane import qchem
import sys
from data import *
from nn import *
from utils import *
from vqe import *
print("Successfully imported Dependencies!")

Successfully imported Dependencies!


In [2]:
# Main Script functions:

import math

def R_from_axis_angle(axis, angle):
    """ Find the rotation matrix R(K, theta) 
        @param axis (K): axis of rotation and 
        @param theta: rotation angle in degrees
        @return R: rotation matrix (3x3 numpy array)
    """
    sina = math.sin(angle * np.pi/180) # convert to radians
    cosa = math.cos(angle * np.pi/180) # convert to radians
    axis = unit_vector(axis[:3])
    kx = axis[0]; ky = axis[1]; kz = axis[2]
    # rotation matrix around unit vector
    
    R = np.array([
                [(kx**2)*(1-cosa) + cosa, kx*ky*(1-cosa)-kz*sina, kx*kz*(1-cosa)+ky*sina],
                [kx*ky*(1-cosa)+kz*sina, (ky**2)*(1-cosa) + cosa, ky*kz*(1-cosa)-kx*sina],
                [kx*kz*(1-cosa)-ky*sina, ky*kz*(1-cosa)+kx*sina, (kz**2)*(1-cosa) + cosa]
                ])
    
    return R

def rotate_and_translate_water(x, axis, angle, transl_vec):
    rotation_matrix = R_from_axis_angle(axis, angle)
    new_x = np.array([0.0, 0.0, 0.0,0.0, 0.0, 0.0,0.0, 0.0, 0.0], requires_grad=True)
    
    # Rotate each vertex
    point1 = x[0:3]
    point1_rot = np.dot(rotation_matrix, point1) # Ordering matters. First rotate, then translate.
    point1_rot += transl_vec
    new_x[0:3] = point1_rot
    
    point2 = x[3:6]
    point2_rot = np.dot(rotation_matrix, point2)
    point2_rot += transl_vec
    new_x[3:6] = point2_rot
    
    point3 = x[6:9]
    point3_rot = np.dot(rotation_matrix, point3)
    point3_rot += transl_vec
    new_x[6:9] = point3_rot
    
    return new_x

# Factor to convert from Bohrs to Angstroms
bohr_angs = 0.529177210903

def build_xyz(r, angle): # prepares an equilateral triagnle's coordinates of H3+!
    r = r/bohr_angs
    x = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    
    # O at (0,0,0)
    x[0] = 0.0
    x[1] = 0.0
    x[2] = 0.0
    
    # H1 at (r*cos(angle),r*sin(angle),0)
    x[3] = r * np.cos(angle * np.pi/180)
    x[4] = r * np.sin(angle * np.pi/180)
    x[5] = 0.0
    
    # H2 at (r, 0, 0)
    x[6] = r
    x[7] = 0.0
    x[8] = 0.0
    
    # rotate water:
    new_x = rotate_and_translate_water(x, axis, rot_angle, transl_vec)
    
    return new_x

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2':

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return (np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)) * 180/math.pi) #convert to degrees

In [86]:
# Load in Neural Network Parameters:

nn_layers = [40, 40]
optimizer = optax.adam(0.01)
n_steps = 4000 # original code used 2000
n_params = 2 # number of z-matrix parameters

In [4]:
# Load in molecule data:

symbols = ["O", "H", "H"]
charge = 0
active_electrons = 8
active_orbitals = 6

# Default rotation and translation parameters of molecule in 3D-Space:
axis = np.array([1, -5, 13]) # any set of numbers, bc it will be normalized to unit vector
rot_angle = 114 # in degrees
transl_vec = np.array([0.0000,0.7581,-0.5086]) # keep in atomic units


# Other stats:
wires = active_orbitals * 2 # assumes the STO-3G basis, b/c only works for the Jordan-Wigner mapping

# Making a sparse Hamiltonian:
def H_sparse(r, theta):
    global symbols, charge,  active_electrons, active_orbitals, wires # import in global variables
    Hamiltonian = qml.qchem.molecular_hamiltonian(symbols, build_xyz(r, theta), charge=charge,
                                                  active_electrons = active_electrons,
                                                  active_orbitals = active_orbitals
                                                 )[0]  
    return qml.SparseHamiltonian(Hamiltonian.sparse_matrix(), wires=range(wires))

# r_train, theta_train

In [5]:
# Generate training z-matrix parameters:

'''
IMPORTANT:

Make array or not array decision when which form inputs are fed into
jax.numpy() neural networks, which are implemented as jax.numpy (jnp)
matrix and arrays
'''

N_train = 3
# Enter all radial lengths in units of angstroms
# Originally: r_train = [[x] for x in np.linspace(0.2, 2.0, N_train)]
r_train = [x for x in np.linspace(0.2, 2.0, N_train)]

# Enter all angular widths in units of degrees
# Originally: theta_train = [[x] for x in np.linspace(60, 130, N_train)]
theta_train = [x for x in np.linspace(60, 130, N_train)]

In [6]:
# def exact_diag_data_sparse(H, samples, weight, spin, k=None): # g stands for geometry, s stands for state
#     """Generates data with exact diagonalization of the Hamiltonian (assumes the generated Hamiltonian is sparse)"""
#     training_g, training_s = [], []
#     for r in tqdm(samples):
#         h = H(r).sparse_matrix()
#         k = 6 if k is None else k
#         v, w = allowed_vec_val_sparse(h, weight, spin, k=k) # 'allowed_vec_val_sparse' is in line 125 above
#         # allowed_vec_val_sparse() returns v(list of eigenvectors) and w(list of eigenvalues) that
        
#         min_index = np.where(w == min(w)) # in list of all eigenstates + corresponding eigenvalues, return index of lowest eigenvalue
#         training_g.append(jnp.array(r)) # append the param "r" that we're shifting by to create sample
#         training_s.append(jnp.array(v.T[min_index][0])) # append ground state of Hamiltonian (eigenvector of H-matrix)
#     return jnp.array(training_g), jnp.array(training_s)

# g_data, s_data = exact_diag_data_sparse(H_sparse, samples, mol.active_electrons, 0)

def exact_diag_data_sparse(H, r_train, theta_train, weight, spin, k=None): # g stands for geometry, s stands for state
    """Generates data with exact diagonalization of the Hamiltonian (assumes the generated Hamiltonian is sparse)"""
    training_w, training_s = [], []
    for r, theta in zip(r_train, theta_train):
        h = H(r,theta).sparse_matrix()
        k = 6 if k is None else k
        v, w = allowed_vec_val_sparse(h, weight, spin, k=k) # 'allowed_vec_val_sparse' is in line 125 above
        # allowed_vec_val_sparse() returns v(list of eigenvectors) and w(list of eigenvalues) that
        
        min_index = np.where(w == min(w)) # in list of all eigenstates + corresponding eigenvalues, return index of lowest eigenvalue
        training_w.append(jnp.array(w[min_index])) # append the param "r" that we're shifting by to create sample
        training_s.append(jnp.array(v.T[min_index][0])) # append ground state of Hamiltonian (eigenvector of H-matrix)
    return jnp.array(training_w), jnp.array(training_s)

In [7]:
# Generate training quantum wavefunctions:
'''
Notes:
-> Extract eigenvalues which original function doesn't do, and see if it looks correct
Done, Checked for water's values, and it does! Now we know that the generated training wavefunctions
are correct for each pair of z-matrix parameters

->Test out ordering of sparse-matrix:
Done, Saw that it mattered, keep in same ordering as function, because we have to convert it back to an array
'''
# for me: reference extract_data() and in it, coupe lines down of model_energy() call in main script at the very
# beginning to see how they check the ground-state eigenvalue -> check these values for this training data by
# plotting it out!

# Test out ordering of sparse-matrix

eigenval_train, eigenvec_train = exact_diag_data_sparse(H_sparse, r_train, theta_train, active_electrons, 0)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [9]:
eigenval_train

Array([[-55.22178269],
       [-75.01607097],
       [-74.75956212]], dtype=float64)

In [10]:
# def gate_pool(mol, sz=0):
#     """Generates a gate pool and single and double excitations"""
#     singles, doubles = qml.qchem.excitations(electrons=mol.active_electrons, orbitals=2 * mol.active_orbitals, delta_sz=sz)
#     # This comment is confirming this function is still updated -> delta_sz is an actual parameter of the updated function!
#     # Delta-sz specifies the selection rule
    
#     pool1, pool2 = [], []
    
#     # here, we are appending lambda functions
#     for s in singles:
#         pool1.append(lambda p, w=s: qml.SingleExcitation(p, wires=w))
#     for d in doubles:
#         pool2.append(lambda p, w=d: qml.DoubleExcitation(p, wires=w))
        
#         # return pool2, or doubles gates, first in the tuple, because we do adapt-vqe on these gates first!
#     return pool2, pool1

def gate_pool(active_electrons, active_orbitals, sz=0):
    """Generates a gate pool and single and double excitations"""
    singles, doubles = qml.qchem.excitations(electrons=active_electrons, orbitals=2 * active_orbitals, delta_sz=sz)
    # This comment is confirming this function is still updated -> delta_sz is an actual parameter of the updated function!
    # Delta-sz specifies the selection rule
    
    pool1, pool2 = [], []
    
    # here, we are appending lambda functions
    for s in singles:
        pool1.append(lambda p, w=s: qml.SingleExcitation(p, wires=w))
    for d in doubles:
        pool2.append(lambda p, w=d: qml.DoubleExcitation(p, wires=w))
        
        # return pool2, or doubles gates, first in the tuple, because we do adapt-vqe on these gates first!
    return pool2, pool1


# def generate_hf_state(molecule):
#     """Returns the HF state for a molecule"""
#     return qchem.hf_state(molecule.active_electrons, 2 * molecule.active_orbitals)

def generate_hf_state(active_electrons, active_orbitals):
    """Returns the HF state for a molecule"""
    return qchem.hf_state(active_electrons, 2 * active_orbitals)


In [11]:
# Conduct Adapt-VQE procedure for selecting relevant quantum gates in training data:

# Initializes an optimizer for the adaptive procedure
optimizer = qml.GradientDescentOptimizer(stepsize=0.1)
# Initialize device to also run procedure:
dev = qml.device("default.qubit", wires = 2*active_orbitals)
# Define number of steps for optimizing double excitation gates:
num_steps = 20
# variable for wires (used in later code):
wires = 2*active_orbitals

# return all possible single and double excitation gates with a molecule of this size active space:
pool = gate_pool(active_electrons, active_orbitals)

# now, perform Adapt-VQE gate selection algorithm for all possible training z-matrix geometries:
gates = []
for r, theta in zip(r_train, theta_train):
    H_hf = H_sparse(r, theta)
    gates_t, params = batch_adapt_vqe(H_hf, dev, pool,
                                      generate_hf_state(active_electrons, active_orbitals),
                                      optimizer, num_steps, # generate_hf_state() is found in data.py, line 215.    
                                      bar=False, sparse=True, tol=1e-5)
    gates.extend(gates_t) # add all gates for each geometry
gates = list(set(gates))

In [75]:
len(gates)

50

In [87]:
def circuit(gates, wires, initial):
    """Circuit for generating data"""
    def data_generating_circuit(params):
        qml.BasisState(initial, wires=range(wires)) # change is re-define this to be range(wires)
        for p, g in zip(params, gates):
            g(p)
    return data_generating_circuit

def state_circuit(circ, dev):
    """State generating circuit"""
    @qml.qnode(dev, interface="jax") # impotant to have the interface be "jax"?
    def state_circ(params):
        circ(params)
        return qml.state()
    return state_circ

In [88]:
# Functions to return full quantum-state of mplecule based on gate parameters:

# cicuit() returns a circuit lambda function. All we have to do is provide parameters and this creates
# quantum-circuit with parameters applied to selected gates. This circuit is passed into next function, state_circuit()
circ = circuit(gates, wires, generate_hf_state(active_electrons, active_orbitals)) # Circuit ansatz. 'circuit()' is from vqe.py, line 238

# state_circuit() uses above circuit, and makes a qnode out of it with a device, and actually returns a quantum state.
# same thing, tho. In the return of this function, so "state_circ", just pass in parameters of gates and out comes
# a quantum state!
# To note is that the interface should be "jax", ig for optimization of the nn later in the code?
state_circ = state_circuit(circ, dev) # State-generating circuit: 'state_circuit' function pulled it from vqe.py, line 246


In [89]:
# def exact_fidelity_loss(model, x, y): # x=g_data (vector of geometry inputs), y=s_data (vector of corresponding target quantum states)
#     """
#     Exact fidelity loss function for a given neural network (this function's return type will
#     accept different neural network parameters, NN)
    
#     Imran's Note: NN are the neural network parameters
#     """
#     def fn(NN): # from the code in "run.py" we can tell that "NN" is the initial neural network parameters, or "initial_NN_params"
#         samples = len(x)
#         return jnp.real((1/samples) * jnp.sum(batch_exact_fidelity_loss(model)(NN, x, y))) # jnp.sum will sum up the "1-" in 
#             # exact_fidelity_loss_sample to be "N -", but the "1/samples" in this function returns it to the "1" we see in the
#             # official formula for the cost function!
#     return fn

def exact_fidelity_loss(model, x, y): # x=g_data (vector of geometry inputs), y=s_data (vector of corresponding target quantum states)
    """
    Exact fidelity loss function for a given neural network (this function's return type will
    accept different neural network parameters, NN)
    
    Imran's Note: NN are the neural network parameters
    """
    def fn(NN): # from the code in "run.py" we can tell that "NN" is the initial neural network parameters, or "initial_NN_params"
        samples = len(x)
        return jnp.real((1/samples) * jnp.sum(jnp.array([exact_fidelity_loss_sample(model)(NN, g, s) for g, s in zip(x, y)]))) 
            # exact_fidelity_loss_sample to be "N -", but the "1/samples" in this function returns it to the "1" we see in the
            # official formula for the cost function!
    return fn

In [90]:
exact_fidelity_loss(model, g_data, s_data)(initial_NN_params)

Array(0.34616549, dtype=float64)

In [91]:
# Finally, set up functions to begin training neural network!!

# re-shape all z-matrix arrays into an array of jnp-arrays -> call g_data:
g_data = []
for r, theta in zip(r_train, theta_train):
    g_data.append(jnp.array([r, theta]))
   
# define s-data is array of eigenvectors from earlier:
s_data = eigenvec_train
    
# re-define device according to Pennylane documentation on how to work with Jax interface:
dev = qml.device("default.qubit.jax", wires = 2*active_orbitals)

# jax key to initialize neural network values (uses jax framework also)
key = jax.random.PRNGKey(100)

# Specifies the sizes of each of the layers in the feed-forward NN
layer_sizes = [n_params] + nn_layers + [len(gates)]

# Specifies the initial NN params (all set to 0.0)
initial_NN_params = network_params(layer_sizes, key, zero=True) # initial_NN_params is an array of tuples, holding the Matrix/Vector values of each layer for all neuron weights and biases   

# Quantum model
def model(geometry, NN_theta): # NN_theta is an array of tuples, storing (W,b). W is a matrix of weights, and b is a vector of biases
    angles = neural_network(NN_theta, geometry)
    return state_circ(angles)

optimizer = optax.adam(0.01) # re-define optimizer variable to be the optax one that will train on the neural network!
params = {'w': initial_NN_params}
opt_state = optimizer.init(params) # from documentaton -> initialize optimizer with the parameters of the nn you want to join

# Loss function
loss = lambda NN : exact_fidelity_loss(model, g_data, s_data)(NN['w']) # notice 'NN' into this function. Later, we see it's passed as 'params'

# Gradient of loss function
gradient_fn = jax.jit(jax.value_and_grad(loss))

# finally, initialize amount of forward and backward passes of the neural network
steps = n_steps

In [92]:
# Train the neural network.

bar = tqdm(range(steps))
print("Initial loss = {}".format(loss(params))) 

# Performs optimization of the model
for s in bar:
    v, gr = gradient_fn(params) # returns v, which is value, and gr, which is gradient.
    bar.set_description(str(v))

    # Computes the gradient, updates the parameters
    updates, opt_state = optimizer.update(gr, opt_state) # opt_state is the state of the optimizer. updates is the actual updates of the NN params!
    params = optax.apply_updates(params, updates) # what optax updates and returns to you is the updated dictionary that * holds* the parameters?

print("Final loss = {}".format(loss(params))) 

# Optimized model
optimized_model = lambda g : model(g, params['w'])


  0%|          | 0/4000 [00:00<?, ?it/s]

Initial loss = 0.34616549023591525
Initial loss = 0.34616549023591525
Final loss = 0.24187184796948097


In [108]:
# Testing out the neural network:

r = 1.01
theta = 104.5

test_sample = jnp.array([r, theta])
H_test = H_sparse(r, theta)
model_val = optimized_model(test_sample)
m =  np.real(np.dot(np.conj(model_val), H_test.sparse_matrix() @ model_val))
m

-74.9476520763531

In [112]:
theta_arr = []
m_arr = []

theta_base = 90.0
for i in range(20):
    theta = theta_base+i
    test_sample = jnp.array([r, theta])
    H_test = H_sparse(r, theta)
    model_val = optimized_model(test_sample)
    m =  np.real(np.dot(np.conj(model_val), H_test.sparse_matrix() @ model_val))
    print("Z-matrix params for Water:", r, "and", theta)
    print("Energy Eigenvalue:", m, "Ha")
    print("")

Z-matrix params for Water: 1.01 and 90.0
Energy Eigenvalue: -74.91459638368569 Ha

Z-matrix params for Water: 1.01 and 91.0
Energy Eigenvalue: -74.91802765494346 Ha

Z-matrix params for Water: 1.01 and 92.0
Energy Eigenvalue: -74.92128072456106 Ha

Z-matrix params for Water: 1.01 and 93.0
Energy Eigenvalue: -74.92435742134862 Ha

Z-matrix params for Water: 1.01 and 94.0
Energy Eigenvalue: -74.92725956606647 Ha

Z-matrix params for Water: 1.01 and 95.0
Energy Eigenvalue: -74.9299889747712 Ha

Z-matrix params for Water: 1.01 and 96.0
Energy Eigenvalue: -74.93254746190526 Ha

Z-matrix params for Water: 1.01 and 97.0
Energy Eigenvalue: -74.93493684316674 Ha

Z-matrix params for Water: 1.01 and 98.0
Energy Eigenvalue: -74.93715893817529 Ha

Z-matrix params for Water: 1.01 and 99.0
Energy Eigenvalue: -74.9392155729653 Ha

Z-matrix params for Water: 1.01 and 100.0
Energy Eigenvalue: -74.94110858231805 Ha

Z-matrix params for Water: 1.01 and 101.0
Energy Eigenvalue: -74.94283981195764 Ha

Z-ma